# Sentimen Analysis

## importing library


In [1]:
!pip install google-play-scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.4 MB/s eta 0:00:00


In [2]:
# Mengimpor pustaka google_play_scraper untuk mengakses ulasan dan informasi aplikasi dari Google Play Store.
from google_play_scraper import app, reviews, Sort, reviews_all

In [3]:
import pandas as pd
pd.options.mode.chained_assignment = None # Menonaktifkan peringatan chaining
import numpy as np
seed = 0
np.random.seed(seed) # Mengatur seed untuk reproduktibilitas
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import re
import string # Berisi konstanta string seperti tanda baca
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory # Penghilang imbuhan kata dalam bahasa indonesia
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory # Menghapus kata-kata yang tidak penting

from wordcloud import WordCloud # Membuat visualisasi berbentuk awan kata dari teks
nltk.download('punkt') # download dataset yang diperlukan untuk kebutuhan tokenisasi teks
nltk.download('stopwords') # mengunduh dataset yang berisi daftar kata kata berhenti dalam berbagai bahasa


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.6 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Scrapping


In [4]:
from itertools import count
# Mengimpor pustaka google_play_scraper untuk mengakses ulasan dan informasi aplikasi dari Google Play Store.
from google_play_scraper import app, reviews_all, Sort

scrapreview = reviews_all(
    'com.byu.id', # ID Aplikasi
    lang='id',    # bahasa ulasan dengan default bahasa inggris
    country='id', # Negara dengan default amerika
    sort=Sort.MOST_RELEVANT,  # Urutan ulasan
    count=10000 # Jumlah ulasan yang diambil
)

In [5]:
import csv

with open('ulasan_aplikasi.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Review']) # Menulis header kolom
    for review in scrapreview:
        writer.writerow([review['content']]) #Menulis konten ulasan ke dalam file csv

## Data Preprocessing


In [6]:
df = pd.DataFrame(scrapreview)
df.shape
df.head()
df.to_csv('ulasan_aplikasi.csv', index=False)

# Membuat Data Frame dari hasil scrapreview
df = pd.DataFrame(scrapreview)
jumlah_ulasan, jumlah_kolom = df.shape

In [7]:
df.shape

(148709, 11)

In [8]:
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,7e64ebf7-4bd7-4155-a3af-d1a377193556,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Dari awal pake by.u jaringannya bagus bgt, eh ...",1,307,1.56.0,2024-08-16 15:22:21,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2024-08-16 16:46:35,1.56.0
1,9d69b674-a5fd-47a3-bc3a-933f92cb814e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"jujur aplikasinya bagus, kartu bagus, cuman ad...",1,104,1.56.0,2024-08-20 16:40:41,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2024-08-20 16:55:52,1.56.0
2,a468cdd6-203b-4360-97a7-31ea50fb2ced,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Aplikasi Error, saat mau mengirim tangkapan la...",2,31,1.56.0,2024-08-17 14:58:40,"Hi, Kak mohon maaf banget udah bikin ga nyaman...",2024-08-17 19:47:43,1.56.0
3,e1655b9e-9e49-4f6f-8d34-55ea8a80dd9a,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Sinyal bagus, kadang sesekali kurang bagusnya ...",3,31,1.56.0,2024-08-20 16:01:55,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2024-08-20 16:55:06,1.56.0
4,c382e959-77ad-415a-8d76-16d701f1bdda,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Kartu by.u ini sangat bagus jaringan juga cepa...,5,47,1.56.0,2024-08-28 02:26:39,"Hallo Kak Bayu, makasih rating dan feedbacknya...",2024-08-28 20:55:19,1.56.0


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148709 entries, 0 to 148708
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   reviewId              148709 non-null  object        
 1   userName              148709 non-null  object        
 2   userImage             148709 non-null  object        
 3   content               148708 non-null  object        
 4   score                 148709 non-null  int64         
 5   thumbsUpCount         148709 non-null  int64         
 6   reviewCreatedVersion  125288 non-null  object        
 7   at                    148709 non-null  datetime64[ns]
 8   replyContent          130485 non-null  object        
 9   repliedAt             130485 non-null  datetime64[ns]
 10  appVersion            125288 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 12.5+ MB


In [10]:
df = df.dropna()
df.isnull().sum()

,0
reviewId,0
userName,0
userImage,0
content,0
score,0
thumbsUpCount,0
reviewCreatedVersion,0
at,0
replyContent,0
repliedAt,0


In [11]:
df.duplicated().sum()

0

### Fungsi-fungsi yang disediakan akan sangat berguna untuk langkah-langkah preprocessing teks. Mari kita jelaskan masing-masing fungsi dengan lebih singkat.

* cleaningText(text): Membersihkan teks dengan menghapus mention, hashtag, RT (retweet), tautan (link), angka, dan tanda baca. Selain itu, karakter newline diganti dengan spasi dan spasi ekstra pada awal dan akhir teks dihapus.

* casefoldingText(text): Mengonversi semua karakter dalam teks menjadi huruf kecil (lowercase) untuk membuat teks menjadi seragam.
tokenizingText(text): Memecah teks menjadi daftar kata atau token. Ini membantu dalam mengurai teks menjadi komponen-komponen dasar untuk analisis lebih lanjut.

* filteringText(text): Menghapus kata-kata berhenti (stopwords) dalam teks. Daftar kata-kata berhenti telah diperbarui dengan beberapa kata tambahan.

* stemmingText(text): Menerapkan stemming pada teks, yakni mengurangi kata-kata menjadi bentuk dasarnya. Anda menggunakan pustaka Sastrawi untuk melakukan stemming dalam bahasa Indonesia.
toSentence(list_words): Menggabungkan daftar kata-kata menjadi sebuah kalimat.

In [12]:
def cleaningText(text):
  text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
  text = re.sub(r'#[A-Za-z0-9]+', '', text) # Menghapus hastag
  text = re.sub(r'RT[\s]+', '', text) # Menghapus RT
  text = re.sub(r'https?:\/\/\S+', '', text) # Menghapus tautan
  text = re.sub(r'[0-9+]', ' ', text) # Menghapus Angka
  text = re.sub(r'[^\w\s]', '', text) # Menghapus karakter selain huruf dan angka

  text = text.replace('\n', ' ') # Mengganti karakter newline dengan spasi
  text = text.strip() # Menghapus spasi ekstra di awal dan akhir teks
  text = text.translate(str.maketrans('', '', string.punctuation)) # Menghapus semua tanda baca
  return text

def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
  text = text.lower()
  return text

def filteringText(text): #Menghapus Stopwords dalam teks
  listStopwords = set(stopwords.words('indonesian'))
  listStopwords1 = set(stopwords.words('english'))
  listStopwords.update(listStopwords1)
  listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
  filtered = []
  for txt in text:
    if txt not in listStopwords:
      filtered.append(txt)
  text = filtered
  return text

def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Memecah teks menjadi daftar kata
    words = text.split()

    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in words]

    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

In [13]:
slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal"}
def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

In [14]:
!pip install nltk
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
def tokenizingText(text):
    tokens = word_tokenize(text)
    return tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
# Membersihkan teks dan menyimpannya di kolom 'text_clean'
df['text_clean'] = df['content'].apply(cleaningText)

# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
df['text_casefoldingText'] = df['text_clean'].apply(casefoldingText)

# Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
df['text_slangwords'] = df['text_casefoldingText'].apply(fix_slangwords)

# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
df['text_tokenizingText'] = df['text_slangwords'].apply(tokenizingText)

# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
df['text_stopword'] = df['text_tokenizingText'].apply(filteringText)

# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
df['text_akhir'] = df['text_stopword'].apply(toSentence)

In [16]:
df

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir
0,7e64ebf7-4bd7-4155-a3af-d1a377193556,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Dari awal pake by.u jaringannya bagus bgt, eh ...",1,307,1.56.0,2024-08-16 15:22:21,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2024-08-16 16:46:35,1.56.0,Dari awal pake byu jaringannya bagus bgt eh bb...,dari awal pake byu jaringannya bagus bgt eh bb...,dari awal pake byu jaringannya bagus banget eh...,"[dari, awal, pake, byu, jaringannya, bagus, ba...","[pake, byu, jaringannya, bagus, banget, eh, bb...",pake byu jaringannya bagus banget eh bbrp kede...
1,9d69b674-a5fd-47a3-bc3a-933f92cb814e,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"jujur aplikasinya bagus, kartu bagus, cuman ad...",1,104,1.56.0,2024-08-20 16:40:41,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2024-08-20 16:55:52,1.56.0,jujur aplikasinya bagus kartu bagus cuman ada ...,jujur aplikasinya bagus kartu bagus cuman ada ...,jujur aplikasinya bagus kartu bagus cuman ada ...,"[jujur, aplikasinya, bagus, kartu, bagus, cuma...","[jujur, aplikasinya, bagus, kartu, bagus, cuma...",jujur aplikasinya bagus kartu bagus cuman kelu...
2,a468cdd6-203b-4360-97a7-31ea50fb2ced,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Aplikasi Error, saat mau mengirim tangkapan la...",2,31,1.56.0,2024-08-17 14:58:40,"Hi, Kak mohon maaf banget udah bikin ga nyaman...",2024-08-17 19:47:43,1.56.0,Aplikasi Error saat mau mengirim tangkapan lay...,aplikasi error saat mau mengirim tangkapan lay...,aplikasi error saat mau mengirim tangkapan lay...,"[aplikasi, error, saat, mau, mengirim, tangkap...","[aplikasi, error, mengirim, tangkapan, layar, ...",aplikasi error mengirim tangkapan layar live n...
3,e1655b9e-9e49-4f6f-8d34-55ea8a80dd9a,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Sinyal bagus, kadang sesekali kurang bagusnya ...",3,31,1.56.0,2024-08-20 16:01:55,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2024-08-20 16:55:06,1.56.0,Sinyal bagus kadang sesekali kurang bagusnya t...,sinyal bagus kadang sesekali kurang bagusnya t...,sinyal bagus kadang sesekali kurang bagusnya t...,"[sinyal, bagus, kadang, sesekali, kurang, bagu...","[sinyal, bagus, kadang, bagusnya, tp, gpp, knp...",sinyal bagus kadang bagusnya tp gpp knp skrg g...
4,c382e959-77ad-415a-8d76-16d701f1bdda,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Kartu by.u ini sangat bagus jaringan juga cepa...,5,47,1.56.0,2024-08-28 02:26:39,"Hallo Kak Bayu, makasih rating dan feedbacknya...",2024-08-28 20:55:19,1.56.0,Kartu byu ini sangat bagus jaringan juga cepat...,kartu byu ini sangat bagus jaringan juga cepat...,kartu byu ini sangat bagus jaringan juga cepat...,"[kartu, byu, ini, sangat, bagus, jaringan, jug...","[kartu, byu, bagus, jaringan, cepat, buka, apl...",kartu byu bagus jaringan cepat buka aplikasi b...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142092,cd5f5eaf-f7fa-4619-b4ec-4f8f627d0cbe,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Oke,5,0,1.0.318 - by.U,2019-10-25 12:44:20,Hi kak makasi yah buat support nya :),2019-10-28 02:42:49,1.0.318 - by.U,Oke,oke,oke,[oke],[oke],oke
142096,5d3c977c-f441-4278-be52-69d670684a58,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Ga bsa dafatar via fb,1,1,1.0.313,2019-10-19 09:02:35,"hai hai, untuk troubleshoot solution bisa hubu...",2019-10-19 11:22:14,1.0.313,Ga bsa dafatar via fb,ga bsa dafatar via fb,ga bsa dafatar via fb,"[ga, bsa, dafatar, via, fb]","[bsa, dafatar, via, fb]",bsa dafatar via fb
142097,a81ed629-6cc7-4a68-8d9f-fe527924dab2,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Gak bisa sign up atau log in,1,1,1.0.313,2019-10-18 19:43:03,"Hi kak,\nNindy mohon maap yaaa, kita emang sem...

In [17]:
import csv
import requests
from io import StringIO

# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_positive[row[0]] = int(row[1])
        # Menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive
else:
    print("Failed to fetch positive lexicon data")

# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()

response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub

if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma

    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")

In [18]:
# Fungsi untuk menentukan polaritas sentimen dari tweet

def sentiment_analysis_lexicon_indonesia(text):
    #for word in text:

    score = 0
    # Inisialisasi skor sentimen ke 0

    for word in text:
        # Mengulangi setiap kata dalam teks

        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
            # Jika kata ada dalam kamus positif, tambahkan skornya ke skor sentimen

    for word in text:
        # Mengulangi setiap kata dalam teks (sekali lagi)

        if (word in lexicon_negative):
            score = score + lexicon_negative[word]
            # Jika kata ada dalam kamus negatif, kurangkan skornya dari skor sentimen

    polarity=''
    # Inisialisasi variabel polaritas

    if (score >= 0):
        polarity = 'positive'
        # Jika skor sentimen lebih besar atau sama dengan 0, maka polaritas adalah positif
    elif (score < 0):
        polarity = 'negative'
        # Jika skor sentimen kurang dari 0, maka polaritas adalah negatif

    # else:
    #     polarity = 'neutral'
    # Ini adalah bagian yang bisa digunakan untuk menentukan polaritas netral jika diperlukan

    return score, polarity
    # Mengembalikan skor sentimen dan polaritas teks

In [19]:
results = df['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
df['polarity_score'] = results[0]
df['polarity'] = results[1]
print(df['polarity'].value_counts())

polarity
positive    65203
negative    44302
Name: count, dtype: int64


## Modeling

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X = df['text_akhir']
y = df['polarity']

# Ekstraksi fitur dengan TF-IDF
tfidf = TfidfVectorizer(max_features=200, min_df=17, max_df=0.8 )
X_tfidf = tfidf.fit_transform(X)

# Konversi hasil ekstraksi fitur menjadi dataframe
features_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())

# Menampilkan hasil ekstraksi fitur
features_df

# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

### NaiveBayes

In [21]:
from sklearn.naive_bayes import BernoulliNB

# Membuat objek model Naive Bayes (Bernoulli Naive Bayes)
naive_bayes = BernoulliNB()

# Melatih model Naive Bayes pada data pelatihan
naive_bayes.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_nb = naive_bayes.predict(X_train.toarray())
y_pred_test_nb = naive_bayes.predict(X_test.toarray())

# Evaluasi akurasi model Naive Bayes
accuracy_train_nb = accuracy_score(y_pred_train_nb, y_train)
accuracy_test_nb = accuracy_score(y_pred_test_nb, y_test)

# Menampilkan akurasi
print('Naive Bayes - accuracy_train:', accuracy_train_nb)
print('Naive Bayes - accuracy_test:', accuracy_test_nb)

Naive Bayes - accuracy_train: 0.7829779462125017
Naive Bayes - accuracy_test: 0.7832519063056481


### Random Forest

In [22]:
from sklearn.ensemble import RandomForestClassifier

# Membuat objek model Random Forest
random_forest = RandomForestClassifier()

# Melatih model Random Forest pada data pelatihan
random_forest.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_rf = random_forest.predict(X_train.toarray())
y_pred_test_rf = random_forest.predict(X_test.toarray())

# Evaluasi akurasi model Random Forest
accuracy_train_rf = accuracy_score(y_pred_train_rf, y_train)
accuracy_test_rf = accuracy_score(y_pred_test_rf, y_test)

# Menampilkan akurasi
print('Random Forest - accuracy_train:', accuracy_train_rf)
print('Random Forest - accuracy_test:', accuracy_test_rf)

Random Forest - accuracy_train: 0.9730149308250765
Random Forest - accuracy_test: 0.8949363042783435


### Logistic Regression

In [23]:
from sklearn.linear_model import LogisticRegression

# Membuat objek model Logistic Regression
logistic_regression = LogisticRegression()

# Melatih model Logistic Regression pada data pelatihan
logistic_regression.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_lr = logistic_regression.predict(X_train.toarray())
y_pred_test_lr = logistic_regression.predict(X_test.toarray())

# Evaluasi akurasi model Logistic Regression pada data pelatihan
accuracy_train_lr = accuracy_score(y_pred_train_lr, y_train)

# Evaluasi akurasi model Logistic Regression pada data uji
accuracy_test_lr = accuracy_score(y_pred_test_lr, y_test)

# Menampilkan akurasi
print('Logistic Regression - accuracy_train:', accuracy_train_lr)
print('Logistic Regression - accuracy_test:', accuracy_test_lr)

Logistic Regression - accuracy_train: 0.9108145746769554
Logistic Regression - accuracy_test: 0.9090909090909091


### Decision Tree

In [26]:
from sklearn.tree import DecisionTreeClassifier

# Membuat objek model Decision Tree
decision_tree = DecisionTreeClassifier()

# Melatih model Decision Tree pada data pelatihan
decision_tree.fit(X_train.toarray(), y_train)

# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_dt = decision_tree.predict(X_train.toarray())
y_pred_test_dt = decision_tree.predict(X_test.toarray())

# Evaluasi akurasi model Decision Tree
accuracy_train_dt = accuracy_score(y_pred_train_dt, y_train)
accuracy_test_dt = accuracy_score(y_pred_test_dt, y_test)

print('Decision Tree - accuracy_train:', accuracy_train_dt)
print('Decision Tree - accuracy_test:', accuracy_test_dt)

Decision Tree - accuracy_train: 0.9730149308250765
Decision Tree - accuracy_test: 0.8635678736130771


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import xgboost as xgb

# Asumsikan df adalah DataFrame yang sudah didefinisikan sebelumnya dengan kolom 'text_akhir' dan 'polarity'
X = df['text_akhir']
y = df['polarity']

# Mengubah label menjadi numerik
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Ekstraksi fitur dengan TF-IDF
tfidf = TfidfVectorizer(max_features=200, min_df=17, max_df=0.8)
X_tfidf = tfidf.fit_transform(X)

# Konversi hasil ekstraksi fitur menjadi dataframe (opsional, jika ingin melihat fitur)
features_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())

# Menampilkan hasil ekstraksi fitur
print(features_df.head())

# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Konversi data latih dan uji menjadi DMatrix, format yang dioptimalkan oleh XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set parameter untuk model XGBoost
params = {
    'objective': 'binary:logistic',  # Jika klasifikasi biner, sesuaikan jika multiclass
    'max_depth': 6,
    'eta': 0.1,  # Learning rate
    'eval_metric': 'logloss',  # Metode evaluasi
}

# Melatih model XGBoost
num_boost_round = 100
bst = xgb.train(params, dtrain, num_boost_round)

# Melakukan prediksi pada data uji
y_pred_proba = bst.predict(dtest)
y_pred = [1 if y > 0.5 else 0 for y in y_pred_proba]  # Thresholding

# Evaluasi model dengan akurasi
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')


       aja  aktivasi  aman  aneh       apk  aplikasi  aplikasinya       app  \
0  0.24268       0.0   0.0   0.0  0.000000  0.000000     0.000000  0.190102   
1  0.00000       0.0   0.0   0.0  0.000000  0.145604     0.190506  0.000000   
2  0.00000       0.0   0.0   0.0  0.000000  0.128156     0.335355  0.000000   
3  0.00000       0.0   0.0   0.0  0.373716  0.000000     0.000000  0.000000   
4  0.00000       0.0   0.0   0.0  0.000000  0.342001     0.000000  0.000000   

      bagus   banget  ...  turun  udah       udh  ulang  unlimited  update  \
0  0.212957  0.22876  ...    0.0   0.0  0.162369    0.0        0.0     0.0   
1  0.261919  0.00000  ...    0.0   0.0  0.000000    0.0        0.0     0.0   
2  0.000000  0.12382  ...    0.0   0.0  0.000000    0.0        0.0     0.0   
3  0.253298  0.00000  ...    0.0   0.0  0.000000    0.0        0.0     0.0   
4  0.102534  0.00000  ...    0.0   0.0  0.000000    0.0        0.0     0.0   

   via  wifi        yg  youtube  
0  0.0   0.0  0.126277